In [2]:
import pandas as pd

In [10]:
uf_n = 90
folder = f"TTE_uf{uf_n}"
user_split_df = pd.read_json(f'{folder}/TTE_user_split_uf{uf_n}.json', orient='records')
print(user_split_df.head(2))

   user_id                                            train_h  \
0        1  [2018, 1028, 1962, 1035, 938, 3105, 2028, 1961...   
1        2  [349, 380, 1610, 163, 480, 2916, 3418, 589, 45...   

                                             train_c  \
0  [531, 2918, 661, 1246, 2762, 2797, 1287, 914, ...   
1  [1552, 1792, 1372, 648, 2278, 1370, 2353, 2006...   

                                             train_n  \
0  [1265, 1199, 3793, 2763, 3361, 2407, 1617, 139...   
1  [296, 2502, 1394, 2054, 1674, 3552, 2716, 3481...   

                                              test_h  \
0  [531, 2918, 661, 1246, 2762, 2797, 1287, 914, ...   
1  [1552, 1792, 1372, 648, 2278, 1370, 2353, 2006...   

                                              test_c  \
0  [588, 527, 1545, 2398, 1197, 594, 2321, 1029, ...   
1  [442, 1597, 2881, 165, 1801, 368, 459, 1385, 7...   

                                              test_n  \
0  [329, 380, 2712, 1722, 3703, 2683, 10, 904, 20...   
1  [2336, 588, 2

In [4]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', names=['user_id', 'movie_id', 'rating', 'timestamp'], engine='python')
users = pd.read_csv('ml-1m/users.dat', sep='::', names=['user_id', 'gender', 'age', 'occupation', 'zip'], engine='python')
movies = pd.read_csv('ml-1m/movies.dat', sep='::', names=['movie_id', 'title', 'genres'], engine='python', encoding='latin-1')

# Merge ratings with user and movie info
data = pd.merge(ratings, users, on='user_id')
data = pd.merge(data, movies, on='movie_id')

# Sort entire dataset by user and timestamp
data = data.sort_values(['user_id', 'timestamp'])
print("Done")
data.head()

Done


,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
31,1,3186,4,978300019,F,1,10,48067,"Girl, Interrupted (1999)",Drama
22,1,1270,5,978300055,F,1,10,48067,Back to the Future (1985),Comedy|Sci-Fi
27,1,1721,4,978300055,F,1,10,48067,Titanic (1997),Drama|Romance
37,1,1022,5,978300055,F,1,10,48067,Cinderella (1950),Animation|Children's|Musical
24,1,2340,3,978300103,F,1,10,48067,Meet Joe Black (1998),Romance


In [5]:
def transform_TTE_to_io(row):

    def process_history(history): 
        prompt = "%% START HISTORY %%\n"
        for i_film in range(len(history)):
            film = history.iloc[i_film]
            # prompt = prompt + "Movie name: " + film['title']+ "\tRating:" + str(film['rating']*(2.0)) + "\n"
            prompt = prompt + "Movie name: " + film['title']+ "\tRating:" + str(film['rating']) + "\n"
        prompt = prompt + "%% END HISTORY %%\n"
        prompt = "Based on the user's watching history and the film's rating, order the list of candidate films. "+\
                 "Order the films where the first one of the list is the most likely to be watched by the user. "+\
                 "The output should only contain the ordered list of the recommended films bounded by the  "+\
                 "special strings '%% START RECOMMENDED LIST %%' and '%% END LIST %%'. "+\
                 "Here follows the user's history and the list of candidate films.\n"+\
                 prompt
        return prompt

    def process_candidates(candidates):
        prompt = "%% START CANDIDATES %%\n"
        for i_film in range(len(candidates)):
            film = candidates.iloc[i_film]
            prompt = prompt + "Movie name: " + film['title']+ "\n"
        prompt = prompt + "%% END CANDIDATES %%"
        return prompt

    def process_rating(candidates):
        output = "%% START RECOMMENDED LIST %%\n"
        for i_film in range(len(candidates)):
            film = candidates.iloc[i_film]
            output = output + "Movie name: " + film['title']+ "\n"
        output = output + "%% END LIST %%"
        return output

    # print(row.keys()[0])
    user_ratings = data[data['user_id'] == row['user_id']]
    resulting_io = []
    for scope in range(3):  # 0:train, 1:test, 2:eval
        mask = user_ratings['movie_id'].isin(row[row.keys()[1+scope*3]])
        history = user_ratings[mask].sort_values(by=['timestamp'])

        mask = user_ratings['movie_id'].isin(row[row.keys()[2+scope*3]])
        resulting_list = user_ratings[mask].sort_values(by=['rating', 'timestamp'])

        mask = movies['movie_id'].isin(row[row.keys()[3+scope*3]])
        noise = movies[mask]

        candidates = pd.concat([resulting_list, noise], axis=0, ignore_index=True)
        random_candidates = candidates.sample(frac=1, random_state=1).reset_index(drop=True)

        input_prompt = process_history(history)
        input_prompt = input_prompt + process_candidates(random_candidates)
        output_prompt = process_rating(candidates)
        #print(input_prompt,'\n',output_prompt)
        resulting_io.append([input_prompt, output_prompt])
    return resulting_io



transform_TTE_to_io(user_split_df.loc[0])


[["Based on the user's watching history and the film's rating, order the list of candidate films. Order the films where the first one of the list is the most likely to be watched by the user. The output should only contain the ordered list of the recommended films bounded by the  special strings '%% START RECOMMENDED LIST %%' and '%% END LIST %%'. Here follows the user's history and the list of candidate films.\n%% START HISTORY %%\nMovie name: Fargo (1996)\tRating:4\nMovie name: Run Lola Run (Lola rennt) (1998)\tRating:4\nMovie name: Rain Man (1988)\tRating:5\nMovie name: Saving Private Ryan (1998)\tRating:5\nMovie name: Awakenings (1990)\tRating:5\nMovie name: Gigi (1958)\tRating:4\nMovie name: Sound of Music, The (1965)\tRating:5\nMovie name: Driving Miss Daisy (1989)\tRating:4\nMovie name: Bambi (1942)\tRating:4\nMovie name: Mary Poppins (1964)\tRating:5\n%% END HISTORY %%\n%% START CANDIDATES %%\nMovie name: Mystery Men (1999)\nMovie name: Time Bandits (1981)\nMovie name: Escape f

In [6]:
user_split_df['io_samples'] = user_split_df.apply(transform_TTE_to_io, axis=1)

In [7]:
print(user_split_df.iloc[0])

user_id                                                       1
train_h       [2018, 1028, 1962, 1035, 938, 3105, 2028, 1961...
train_c       [531, 2918, 661, 1246, 2762, 2797, 1287, 914, ...
train_n       [1265, 1199, 3793, 2763, 3361, 2407, 1617, 139...
test_h        [531, 2918, 661, 1246, 2762, 2797, 1287, 914, ...
test_c        [588, 527, 1545, 2398, 1197, 594, 2321, 1029, ...
test_n        [329, 380, 2712, 1722, 3703, 2683, 10, 904, 20...
eval_h        [588, 527, 1545, 2398, 1197, 594, 2321, 1029, ...
eval_c        [48, 1566, 1907, 2294, 2355, 783, 745, 2687, 5...
eval_n        [1294, 25, 2396, 1234, 3175, 316, 913, 3751, 2...
io_samples    [[Based on the user's watching history and the...
Name: 0, dtype: object


In [8]:
user_split_io_df = user_split_df.drop(columns=user_split_df.keys()[1:10])
print(user_split_io_df.iloc[0])
user_split_io_df[['train', 'test', 'eval']] = pd.DataFrame(user_split_io_df['io_samples'].tolist(), index=user_split_io_df.index)
user_split_io_df = user_split_io_df.drop(columns='io_samples')
print(user_split_io_df.iloc[0])

user_id                                                       1
io_samples    [[Based on the user's watching history and the...
Name: 0, dtype: object
user_id                                                    1
train      [Based on the user's watching history and the ...
test       [Based on the user's watching history and the ...
eval       [Based on the user's watching history and the ...
Name: 0, dtype: object


In [11]:
user_split_io_df.to_json(f"{folder}/TTE_with_IO_uf{uf_n}.json", orient="records", indent=4)

In [12]:
for split in ["train", "test", "eval"]:
    df = pd.DataFrame()
    df[['input', 'output']] = pd.DataFrame(user_split_io_df[split].tolist(), index=user_split_io_df.index)
    df.to_json(f"{folder}/TTE_with_IO_{split}.json", orient="records", indent=4)
